# Análise exploratória e testes amostral - linha 685

**TODO:**
- (1) Tabela pré-reprocessamento
- (2) Gráficos da análise exploratória
- (3) Tabela pós-reprocessamento

Preparar o ambiente:

In [1]:
#!pip install matplotlib
import basedosdados as bd
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import timedelta, datetime

pd.set_option('display.max_rows', 25)

# Paths
import os
import sys
from pathlib import Path 

current_path = Path().resolve().parent

paths = dict()
paths["raw"] = current_path / 'data' / 'raw'
paths["treated"] = current_path / 'data' / 'treated'
paths["output"] = current_path / 'data' / 'output'
paths["figures"] = current_path / 'data' / 'figures'
paths["scripts"] = current_path / 'scripts'


for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)      

if paths["scripts"] not in sys.path:
    sys.path.append(str(paths["scripts"]))
 
from graphs import *
from categorize_trips import *

# Seta o projeto a ser cobrado
bd.config.billing_project_id = "rj-smtr-dev"

# Seta as configurações a serem usadas
bd.config.project_config_path = "C:/Users/igorl/.basedosdados/"
# bd.config.project_config_path = "D:\\basedosdados\\dev"


# (1) Tabela pré-reprocessamento

- Tabela de viagens identificadas da amostra: `output/analise_amostra_pre_solucao.xlsx`

> data	servico	sentido	id_veiculo	datetime_partida_amostra	datetime_chegada_amostra	datetime_partida_apuracao	datetime_chegada_apuracao	status

In [2]:
servico = "685"

#### 1 - Importar e tratar os dados da amostra

In [3]:
amostra = pd.read_excel('../data/raw/Apuracao de Viagens D+1 - Analise de dados  (1).xlsx',
                        sheet_name='Rel. Conecta - Mapa corrido')

amostra
# padronizar as colunas da amostra 
amostra = amostra.rename(
    columns={
             "Saída": "hora_inicio",
             "Linha": "servico",
             "Veículo": "id_veiculo",
             "Cheg.": "hora_fim",
             "Sentido": "sentido"
             }
    )

# # setar tipos de dados e criar colunas com o datetime
amostra['servico'] = amostra['servico'].astype(str)
amostra['data'] = '2023-05-31' # segundo informações em outra aba na planilha da amostra
amostra['data'] = amostra['data'].astype(str)
amostra['hora_inicio'] = amostra['hora_inicio'].astype(str)
amostra['datetime_partida'] = pd.to_datetime(amostra['data'] + ' ' + amostra['hora_inicio'])
amostra['hora_fim'] = amostra['hora_fim'].astype(str)
amostra['datetime_chegada'] = pd.to_datetime(amostra['data'] + ' ' + amostra['hora_fim'])

amostra_tratada = amostra[['data','servico','id_veiculo','sentido','datetime_partida','datetime_chegada']]
amostra_tratada

,data,servico,id_veiculo,sentido,datetime_partida,datetime_chegada
0,2023-05-31,685,B44529,I,2023-05-31 05:35:00,2023-05-31 06:46:00
1,2023-05-31,685,B44529,V,2023-05-31 06:47:00,2023-05-31 07:58:00
2,2023-05-31,685,B44529,I,2023-05-31 08:07:00,2023-05-31 09:37:00
3,2023-05-31,685,B44529,V,2023-05-31 09:42:00,2023-05-31 10:59:00
4,2023-05-31,685,B44529,I,2023-05-31 11:12:00,2023-05-31 12:48:00
...,...,...,...,...,...,...
127,2023-05-31,685,C44667,V,2023-05-31 08:33:00,2023-05-31 09:44:00
128,2023-05-31,685,C44667,I,2023-05-31 10:11:00,2023-05-31 11:47:00
129,2023-05-31,685,C44667,V,2023-05-31 11:58:00,2023-05-31 13:18:00
130,2023-05-31,685,C44667,I,2023-05-31 16:52:00,2023-05-31 18:34:00


In [4]:
veiculos = amostra_tratada['id_veiculo'].unique()

print("A quantidade total de viagens na amostra é:", amostra_tratada.shape[0])
print("Os dados da amostra são referentes ao dia 31/05/2023")
print("A amostra tem dados dos seguintes veículos:", veiculos)

A quantidade total de viagens na amostra é: 132
Os dados da amostra são referentes ao dia 31/05/2023
A amostra tem dados dos seguintes veículos: ['B44529' 'B44532' 'B44535' 'B44578' 'B44582' 'B44584' 'B44619' 'B44630'
 'B44642' 'B44648' 'B44653' 'B44659' 'B44661' 'B44671' 'B44672' 'B44673'
 'B44674' 'B44677' 'B44687' 'B44688' 'B44689' 'C44505' 'C44625' 'C44656'
 'C44667']


#### 2 - Remover viagens sobrepostas da amostra

Esta etapa classifica como "Viagem inválida - sobreposição de viagem" aquelas viagens em que um mesmo `id_veiculo` realiza múltiplas viagens em um mesmo intervalo de tempo (`datetime_partida` e `datetime_chegada`).

No caso de duas ou mais viagens sobrepostas, apenas uma será classificada nas demais etapas de classificação do notebook.

In [5]:
amostra_deduplicada = remove_overlapping_trips(amostra_tratada)
amostra_deduplicada

,data,servico,id_veiculo,sentido,datetime_partida,datetime_chegada,status
0,2023-05-31,685,B44529,I,2023-05-31 05:35:00,2023-05-31 06:46:00,NaN
1,2023-05-31,685,B44529,V,2023-05-31 06:47:00,2023-05-31 07:58:00,NaN
2,2023-05-31,685,B44529,I,2023-05-31 08:07:00,2023-05-31 09:37:00,NaN
3,2023-05-31,685,B44529,V,2023-05-31 09:42:00,2023-05-31 10:59:00,NaN
4,2023-05-31,685,B44529,I,2023-05-31 11:12:00,2023-05-31 12:48:00,NaN
...,...,...,...,...,...,...,...
127,2023-05-31,685,C44667,V,2023-05-31 08:33:00,2023-05-31 09:44:00,NaN
128,2023-05-31,685,C44667,I,2023-05-31 10:11:00,2023-05-31 11:47:00,NaN
129,2023-05-31,685,C44667,V,2023-05-31 11:58:00,2023-05-31 13:18:00,NaN
130,2023-05-31,685,C44667,I,2023-05-31 16:52:00,2023-05-31 18:34:00,NaN


In [6]:
count_non_nan = amostra_deduplicada['status'].notna().sum()
count_non_nan

0

Não existem viagens duplicadas na amostra.

### 3 Comparar dados da amostra com os dados das viagens completas

#### 3.1 Consultar se as viagens da amostra foram feitas pelos veículos indicados na amostra

In [7]:
# separar as datas da amostra para fazer a query
datas = amostra_deduplicada['data'].unique()

q = f"""
       SELECT
         id_veiculo, servico_informado, sentido, datetime_partida, datetime_chegada
       FROM
         `rj-smtr.projeto_subsidio_sppo.viagem_completa`
       WHERE
         data = '2023-05-31'
         AND servico_informado = '{servico}'
       """
       
viagem_completa_prod = bd.read_sql(q, from_file=True)
viagem_completa_prod.info() 

# viagem_completa_prod['id_veiculo'] = viagem_completa_prod['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)
viagem_completa_prod['datetime_partida'] = pd.to_datetime(viagem_completa_prod['datetime_partida'])
viagem_completa_prod      


Downloading: 100%|██████████| 90/90 [00:00<00:00, 190.27rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id_veiculo         90 non-null     object        
 1   servico_informado  90 non-null     object        
 2   sentido            90 non-null     object        
 3   datetime_partida   90 non-null     datetime64[ns]
 4   datetime_chegada   90 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 3.6+ KB


,id_veiculo,servico_informado,sentido,datetime_partida,datetime_chegada
0,B44535,685,V,2023-05-31 08:42:46,2023-05-31 09:44:46
1,B44529,685,V,2023-05-31 06:50:08,2023-05-31 07:58:08
2,B44578,685,V,2023-05-31 07:57:58,2023-05-31 09:11:30
3,B44582,685,V,2023-05-31 07:34:42,2023-05-31 08:52:12
4,B44535,685,V,2023-05-31 19:05:44,2023-05-31 20:07:44
...,...,...,...,...,...
85,B44619,685,I,2023-05-31 16:27:11,2023-05-31 17:46:41
86,B44671,685,I,2023-05-31 18:41:52,2023-05-31 19:51:52
87,B44688,685,I,2023-05-31 17:24:51,2023-05-31 18:53:21
88,B44672,685,I,2023-05-31 05:09:59,2023-05-31 06:03:59


Esta etapa cruza dados da amostra com as viagens completas para o serviço 685 de acordo com os seguintes critérios:

- o datetime_partida da viagem que consta na tabela de viagens completas deve estar dentro 
do intervalo de mais ou menos 10 minutos do datetime_partida que consta no amostra.

In [8]:
amostra_classificada = check_trips(amostra_deduplicada, viagem_completa_prod, 10,
                                   "Viagem identificada e já paga")
amostra_classificada

Não existem casos duplicados no cruzamento de dados.


,data,servico_amostra,id_veiculo_amostra,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
0,2023-05-31,685,B44529,V,2023-05-31 09:42:00,2023-05-31 10:59:00,NaN,NaN,NaN,NaN,NaT,NaT
1,2023-05-31,685,B44529,V,2023-05-31 12:55:00,2023-05-31 14:17:00,NaN,NaN,NaN,NaN,NaT,NaT
2,2023-05-31,685,B44529,V,2023-05-31 16:06:00,2023-05-31 17:27:00,NaN,NaN,NaN,NaN,NaT,NaT
3,2023-05-31,685,B44529,V,2023-05-31 19:24:00,2023-05-31 20:35:00,NaN,NaN,NaN,NaN,NaT,NaT
4,2023-05-31,685,B44532,V,2023-05-31 06:35:00,2023-05-31 07:45:00,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
127,2023-05-31,685,C44625,I,2023-05-31 15:58:00,2023-05-31 17:35:00,Viagem identificada e já paga,C44625,685,I,2023-05-31 16:02:23,2023-05-31 17:29:23
128,2023-05-31,685,C44625,V,2023-05-31 17:35:00,2023-05-31 18:57:00,Viagem identificada e já paga,C44625,685,V,2023-05-31 17:38:53,2023-05-31 18:56:53
129,2023-05-31,685,C44667,I,2023-05-31 07:01:00,2023-05-31 08:27:00,Viagem identificada e já paga,C44667,685,I,2023-05-31 07:03:00,2023-05-31 08:22:30
130,2023-05-31,685,C44667,V,2023-05-31 08:33:00,2023-05-31 09:44:00,Viagem identificada e já paga,C44667,685,V,2023-05-31 08:37:30,2023-05-31 09:44:00


In [9]:
count_non_nan = amostra_classificada['status'].notna().sum()
count_non_nan

89

In [10]:
amostra_classificada.to_excel('./../data/treated/classificação_completa.xlsx')

#### 3.2 Dados de Conformidade

Esta etapa cruza dados da amostra e que não foram classificados na etapa anterior com as viagens completas para o serviço 685 de acordo com os seguintes critérios:

- o datetime_partida da viagem que consta na tabela de viagens conformidade deve estar dentro 
do intervalo de mais ou menos 10 minutos do datetime_partida que consta no amostra.

In [11]:
q = f"""
       SELECT
         id_veiculo, servico_informado, sentido, datetime_partida, datetime_chegada
       FROM
         `rj-smtr.projeto_subsidio_sppo.viagem_conformidade`
       WHERE
         data = '2023-05-31'
         AND servico_informado = '{servico}'
       """
       
viagem_conformidade = bd.read_sql(q, from_file=True)
viagem_conformidade.info() 

# viagem_completa_prod['id_veiculo'] = viagem_completa_prod['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)
viagem_conformidade['datetime_partida'] = pd.to_datetime(viagem_conformidade['datetime_partida'])
viagem_conformidade    

Downloading: 100%|██████████| 98/98 [00:00<00:00, 295.18rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id_veiculo         98 non-null     object        
 1   servico_informado  98 non-null     object        
 2   sentido            98 non-null     object        
 3   datetime_partida   98 non-null     datetime64[ns]
 4   datetime_chegada   98 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 4.0+ KB


,id_veiculo,servico_informado,sentido,datetime_partida,datetime_chegada
0,C44505,685,V,2023-05-31 17:57:31,2023-05-31 19:21:01
1,B44535,685,V,2023-05-31 08:42:46,2023-05-31 09:44:46
2,B44582,685,V,2023-05-31 07:34:42,2023-05-31 08:52:12
3,B44584,685,V,2023-05-31 11:52:14,2023-05-31 14:11:45
4,B44535,685,V,2023-05-31 19:05:44,2023-05-31 20:07:44
...,...,...,...,...,...
93,B44677,685,I,2023-05-31 04:50:56,2023-05-31 05:50:26
94,B44674,685,I,2023-05-31 06:11:50,2023-05-31 07:41:20
95,B44659,685,I,2023-05-31 10:51:40,2023-05-31 12:14:10
96,B44661,685,I,2023-05-31 19:05:44,2023-05-31 20:15:14


In [12]:
amostra_classificada_conf = check_trips(amostra_classificada, viagem_conformidade, 10,
                                   "Viagem não atendeu aos critérios de conformidade de percentual de GPS ou do trajeto")
amostra_classificada_conf

Não existem casos duplicados no cruzamento de dados.


,data,servico_amostra,id_veiculo_amostra,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
0,2023-05-31,685,B44529,I,2023-05-31 05:35:00,2023-05-31 06:46:00,Viagem identificada e já paga,B44529,685,I,2023-05-31 05:36:38,2023-05-31 06:44:38
1,2023-05-31,685,B44529,V,2023-05-31 06:47:00,2023-05-31 07:58:00,Viagem identificada e já paga,B44529,685,V,2023-05-31 06:50:08,2023-05-31 07:58:08
2,2023-05-31,685,B44529,I,2023-05-31 08:07:00,2023-05-31 09:37:00,Viagem identificada e já paga,B44529,685,I,2023-05-31 08:09:08,2023-05-31 09:31:38
3,2023-05-31,685,B44529,I,2023-05-31 11:12:00,2023-05-31 12:48:00,Viagem identificada e já paga,B44529,685,I,2023-05-31 11:15:08,2023-05-31 12:36:38
4,2023-05-31,685,B44529,I,2023-05-31 14:37:00,2023-05-31 16:02:00,Viagem identificada e já paga,B44529,685,I,2023-05-31 14:39:08,2023-05-31 15:55:38
...,...,...,...,...,...,...,...,...,...,...,...,...
127,2023-05-31,685,C44656,I,2023-05-31 15:50:00,2023-05-31 17:11:00,NaN,NaN,NaN,NaN,NaT,NaT
128,2023-05-31,685,C44656,V,2023-05-31 17:14:00,2023-05-31 18:50:00,NaN,NaN,NaN,NaN,NaT,NaT
129,2023-05-31,685,C44667,V,2023-05-31 11:58:00,2023-05-31 13:18:00,NaN,NaN,NaN,NaN,NaT,NaT
130,2023-05-31,685,C44667,I,2023-05-31 16:52:00,2023-05-31 18:34:00,NaN,NaN,NaN,NaN,NaT,NaT


In [13]:
amostra_classificada_conf.to_excel('./../data/treated/classificação_conformidade.xlsx')

In [14]:
count_nan_status = amostra_classificada_conf['status'].isna().sum()
print(count_nan_status)

43


As viagens não encontradas na tabela `viagem_completa` também não foram encontradas na tabela `viagem_conformidade` com o parâmetro de 10 minutos para mais ou para menos.

#### 3.3 Verificar dados de GPS (Tabela `gps_sppo`)

In [15]:
amostra_sem_status = amostra_classificada_conf[pd.isna(amostra_classificada_conf['status'])]
amostra_sem_status

,data,servico_amostra,id_veiculo_amostra,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
89,2023-05-31,685,B44529,V,2023-05-31 09:42:00,2023-05-31 10:59:00,NaN,NaN,NaN,NaN,NaT,NaT
90,2023-05-31,685,B44529,V,2023-05-31 12:55:00,2023-05-31 14:17:00,NaN,NaN,NaN,NaN,NaT,NaT
91,2023-05-31,685,B44529,V,2023-05-31 16:06:00,2023-05-31 17:27:00,NaN,NaN,NaN,NaN,NaT,NaT
92,2023-05-31,685,B44529,V,2023-05-31 19:24:00,2023-05-31 20:35:00,NaN,NaN,NaN,NaN,NaT,NaT
93,2023-05-31,685,B44532,V,2023-05-31 06:35:00,2023-05-31 07:45:00,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
127,2023-05-31,685,C44656,I,2023-05-31 15:50:00,2023-05-31 17:11:00,NaN,NaN,NaN,NaN,NaT,NaT
128,2023-05-31,685,C44656,V,2023-05-31 17:14:00,2023-05-31 18:50:00,NaN,NaN,NaN,NaN,NaT,NaT
129,2023-05-31,685,C44667,V,2023-05-31 11:58:00,2023-05-31 13:18:00,NaN,NaN,NaN,NaN,NaT,NaT
130,2023-05-31,685,C44667,I,2023-05-31 16:52:00,2023-05-31 18:34:00,NaN,NaN,NaN,NaN,NaT,NaT


In [16]:
contagem_sentido = amostra_sem_status['sentido_amostra'].value_counts()
contagem_sentido

V    37
I     6
Name: sentido_amostra, dtype: int64

43 das 132 viagens não foram encontradas até aqui, 37 de Volta e 6 de Ida.

Classificar de acordo com o GPS

In [17]:
amostra_com_status = amostra_classificada_conf[pd.notna(amostra_classificada_conf['status'])]
amostra_sem_status = amostra_classificada_conf[pd.isna(amostra_classificada_conf['status'])]
amostra_sem_status

,data,servico_amostra,id_veiculo_amostra,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
89,2023-05-31,685,B44529,V,2023-05-31 09:42:00,2023-05-31 10:59:00,NaN,NaN,NaN,NaN,NaT,NaT
90,2023-05-31,685,B44529,V,2023-05-31 12:55:00,2023-05-31 14:17:00,NaN,NaN,NaN,NaN,NaT,NaT
91,2023-05-31,685,B44529,V,2023-05-31 16:06:00,2023-05-31 17:27:00,NaN,NaN,NaN,NaN,NaT,NaT
92,2023-05-31,685,B44529,V,2023-05-31 19:24:00,2023-05-31 20:35:00,NaN,NaN,NaN,NaN,NaT,NaT
93,2023-05-31,685,B44532,V,2023-05-31 06:35:00,2023-05-31 07:45:00,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
127,2023-05-31,685,C44656,I,2023-05-31 15:50:00,2023-05-31 17:11:00,NaN,NaN,NaN,NaN,NaT,NaT
128,2023-05-31,685,C44656,V,2023-05-31 17:14:00,2023-05-31 18:50:00,NaN,NaN,NaN,NaN,NaT,NaT
129,2023-05-31,685,C44667,V,2023-05-31 11:58:00,2023-05-31 13:18:00,NaN,NaN,NaN,NaN,NaT,NaT
130,2023-05-31,685,C44667,I,2023-05-31 16:52:00,2023-05-31 18:34:00,NaN,NaN,NaN,NaN,NaT,NaT


In [18]:
ids_n_encontrados = amostra_sem_status['id_veiculo_amostra'].unique()
ids_n_encontrados 

array(['B44529', 'B44532', 'B44535', 'B44582', 'B44584', 'B44630',
       'B44642', 'B44648', 'B44653', 'B44659', 'B44661', 'B44671',
       'B44672', 'B44673', 'B44674', 'B44677', 'B44688', 'B44689',
       'C44625', 'C44656', 'C44667'], dtype=object)

In [19]:
data_n_encontradas_gps = pd.Series(pd.to_datetime(amostra_sem_status['data'].unique(), errors='coerce'))
data_n_encontradas_gps = data_n_encontradas_gps.dropna()
data_n_encontradas_gps = data_n_encontradas_gps.dt.strftime('%Y-%m-%d')
data_n_encontradas_gps = data_n_encontradas_gps.values

q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = '2023-05-31'
  AND id_veiculo IN {tuple(ids_n_encontrados)}
"""
          
dados_gps = bd.read_sql(q, from_file=True)
dados_gps.info()

Downloading: 100%|██████████| 38701/38701 [00:03<00:00, 11286.38rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38701 entries, 0 to 38700
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           38701 non-null  object        
 1   servico              38701 non-null  object        
 2   timestamp_gps        38701 non-null  datetime64[ns]
 3   posicao_veiculo_geo  38701 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.2+ MB


In [20]:
def set_status(row, df_check):
    # Filter the df_check by vehicle ID and time range
    filtered_df = df_check[
        (df_check['id_veiculo'] == row['id_veiculo_amostra']) & 
        (df_check['timestamp_gps'] >= row['datetime_partida_amostra']) & 
        (df_check['timestamp_gps'] <= row['datetime_chegada_amostra'])
    ]
    
    # If there are rows in the filtered dataframe, check the condition
    if not filtered_df.empty:
        if all(filtered_df['servico'] == row['servico_amostra']):
            return "Encontrado sinal de GPS para o dia e horário indicados na amostra."
        else:
            unique_services = filtered_df['servico'].unique()
            return "Sinal de GPS encontrado para serviço diferente: {}".format(", ".join(map(str, unique_services)))
    else:
        return "Sem sinal de GPS para o veículo no dia e horário informados"

amostra_sem_status['status'] = amostra_sem_status.apply(lambda row: set_status(row, dados_gps), axis=1)

amostra_sem_status

,data,servico_amostra,id_veiculo_amostra,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
89,2023-05-31,685,B44529,V,2023-05-31 09:42:00,2023-05-31 10:59:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
90,2023-05-31,685,B44529,V,2023-05-31 12:55:00,2023-05-31 14:17:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
91,2023-05-31,685,B44529,V,2023-05-31 16:06:00,2023-05-31 17:27:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
92,2023-05-31,685,B44529,V,2023-05-31 19:24:00,2023-05-31 20:35:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
93,2023-05-31,685,B44532,V,2023-05-31 06:35:00,2023-05-31 07:45:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
127,2023-05-31,685,C44656,I,2023-05-31 15:50:00,2023-05-31 17:11:00,Sinal de GPS encontrado para serviço diferente...,NaN,NaN,NaN,NaT,NaT
128,2023-05-31,685,C44656,V,2023-05-31 17:14:00,2023-05-31 18:50:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
129,2023-05-31,685,C44667,V,2023-05-31 11:58:00,2023-05-31 13:18:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
130,2023-05-31,685,C44667,I,2023-05-31 16:52:00,2023-05-31 18:34:00,Sinal de GPS encontrado para serviço diferente...,NaN,NaN,NaN,NaT,NaT


In [21]:
# juntar todas as linhas e classificar abaixo:
df_final = pd.concat([amostra_com_status, amostra_sem_status], ignore_index=True)
df_final

,data,servico_amostra,id_veiculo_amostra,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
0,2023-05-31,685,B44529,I,2023-05-31 05:35:00,2023-05-31 06:46:00,Viagem identificada e já paga,B44529,685,I,2023-05-31 05:36:38,2023-05-31 06:44:38
1,2023-05-31,685,B44529,V,2023-05-31 06:47:00,2023-05-31 07:58:00,Viagem identificada e já paga,B44529,685,V,2023-05-31 06:50:08,2023-05-31 07:58:08
2,2023-05-31,685,B44529,I,2023-05-31 08:07:00,2023-05-31 09:37:00,Viagem identificada e já paga,B44529,685,I,2023-05-31 08:09:08,2023-05-31 09:31:38
3,2023-05-31,685,B44529,I,2023-05-31 11:12:00,2023-05-31 12:48:00,Viagem identificada e já paga,B44529,685,I,2023-05-31 11:15:08,2023-05-31 12:36:38
4,2023-05-31,685,B44529,I,2023-05-31 14:37:00,2023-05-31 16:02:00,Viagem identificada e já paga,B44529,685,I,2023-05-31 14:39:08,2023-05-31 15:55:38
...,...,...,...,...,...,...,...,...,...,...,...,...
127,2023-05-31,685,C44656,I,2023-05-31 15:50:00,2023-05-31 17:11:00,Sinal de GPS encontrado para serviço diferente...,NaN,NaN,NaN,NaT,NaT
128,2023-05-31,685,C44656,V,2023-05-31 17:14:00,2023-05-31 18:50:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
129,2023-05-31,685,C44667,V,2023-05-31 11:58:00,2023-05-31 13:18:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
130,2023-05-31,685,C44667,I,2023-05-31 16:52:00,2023-05-31 18:34:00,Sinal de GPS encontrado para serviço diferente...,NaN,NaN,NaN,NaT,NaT


In [ ]:
df_final.to_excel('./../data/treated/GPS_classificação_nova.xlsx')

Foi identificado que das 43 viagens não identificadas anteriormente, 11 continham sinais de GPS para outros serviços, como o SVB685.

Quanto às 32 viagens restantes com sinal de GPS para o serviço correto no momento da viagem, 5 não passaram a 500m do ponto inicial ou final e 27 não foram identificadas por desviarem no trajeto na viagem de volta, o que fez com que não houvesse um ponto middle antes ou depois dos pontos de start/end. Vale ressaltar que em algumas viagens foram identificadas mesmo com o desvio na rota, pois registraram um ponto "middle" antes ou depois dos pontos start/end por conta do local que o ônibus enviou o último sinal de GPS antes de entrar no raío de início/fim da viagem.

Estas 32 viagens (com desvio de roda ou fora do raio de 500m) foram classificadas como "Viagem inválida - inconsistência nos dados de GPS".

In [32]:
df_final['status'] = df_final['status'].replace("Encontrado sinal de GPS para o dia e horário indicados na amostra.", "Viagem inválida - inconsistência nos dados de GPS")

In [34]:
df_final.to_excel('./../data/output/analise_amostra_pre_solucao.xlsx')

14-02-2023 Dia antes do problema começar, viagem de volta no trajeto correto

In [28]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2023-02-14"
  AND servico = '685'
"""
       
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.31rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [27]:
q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = '2023-02-14'
  AND id_veiculo = "B44529"
  AND timestamp_gps BETWEEN "2023-02-14T06:27:49" AND "2023-02-14T07:34:49"
"""

gps = bd.read_sql(q, from_file=True)
gps.info() 

Downloading: 100%|██████████| 135/135 [00:00<00:00, 228.81rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           135 non-null    object        
 1   servico              135 non-null    object        
 2   timestamp_gps        135 non-null    datetime64[ns]
 3   posicao_veiculo_geo  135 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 4.3+ KB


In [29]:
create_trip_map(gps, shape_identificado)

15-02-2023, inicio do problema, viagem de volta fora do trajeto.

In [ ]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2023-02-15"
  AND servico = '685'
"""
       
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

In [30]:
q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = '2023-02-15'
  AND id_veiculo = "B44529"
  AND timestamp_gps BETWEEN "2023-02-15T13:08:49" AND "2023-02-15T15:07:49"
"""

gps = bd.read_sql(q, from_file=True)
gps.info() 

Downloading: 100%|██████████| 1185/1185 [00:00<00:00, 2904.41rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1185 entries, 0 to 1184
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           1185 non-null   object        
 1   servico              1185 non-null   object        
 2   timestamp_gps        1185 non-null   datetime64[ns]
 3   posicao_veiculo_geo  1185 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 37.2+ KB


In [31]:
create_trip_map(gps, shape_identificado)

Automatizar mapas

In [27]:
mapa_dados = amostra_sem_status[amostra_sem_status['status'] == 'Viagem inválida - inconsistência nos dados de GPS']
mapa_dados

,data,servico_amostra,id_veiculo_amostra,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
89,2023-05-31,685,B44529,V,2023-05-31 09:42:00,2023-05-31 10:59:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
90,2023-05-31,685,B44529,V,2023-05-31 12:55:00,2023-05-31 14:17:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
91,2023-05-31,685,B44529,V,2023-05-31 16:06:00,2023-05-31 17:27:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
92,2023-05-31,685,B44529,V,2023-05-31 19:24:00,2023-05-31 20:35:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
93,2023-05-31,685,B44532,V,2023-05-31 06:35:00,2023-05-31 07:45:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
125,2023-05-31,685,C44625,V,2023-05-31 07:59:00,2023-05-31 09:09:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
126,2023-05-31,685,C44625,V,2023-05-31 14:24:00,2023-05-31 15:31:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
128,2023-05-31,685,C44656,V,2023-05-31 17:14:00,2023-05-31 18:50:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT
129,2023-05-31,685,C44667,V,2023-05-31 11:58:00,2023-05-31 13:18:00,Encontrado sinal de GPS para o dia e horário i...,NaN,NaN,NaN,NaT,NaT


In [23]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2023-05-31"
  AND servico = '685'
"""
       
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.45rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [26]:

q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = '2023-05-31'
  AND id_veiculo IN {tuple(ids_n_encontrados)}
"""

gps = bd.read_sql(q, from_file=True)
gps.info() 


# q = f"""
# SELECT
#   id_veiculo,
#   servico_informado as servico,
#   timestamp_gps,
#   posicao_veiculo_geo,
#   status_viagem
# FROM
#   `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
# WHERE
#   DATA = '2023-05-31'
#   AND servico_informado = '685'
#   """
# gps = bd.read_sql(q, from_file=True)
# gps.info() 

Downloading: 100%|██████████| 38701/38701 [00:05<00:00, 7616.82rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38701 entries, 0 to 38700
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           38701 non-null  object        
 1   servico              38701 non-null  object        
 2   timestamp_gps        38701 non-null  datetime64[ns]
 3   posicao_veiculo_geo  38701 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.2+ MB


In [28]:
# Para cada linha de mapa_dados
for index, row in mapa_dados.iterrows():
    
    # Filtrar o df gps baseado nas condições fornecidas
    gps_filtrado = gps[(gps['id_veiculo'] == row['id_veiculo_amostra']) & 
                       (gps['timestamp_gps'] >= row['datetime_partida_amostra']) & 
                       (gps['timestamp_gps'] <= row['datetime_chegada_amostra'])]
    
    # Se encontrarmos registros correspondentes
    if not gps_filtrado.empty:
        map = create_trip_map(gps_filtrado, shape_identificado)
        hora_formatada = row['datetime_partida_amostra'].strftime('%Hh%M')
        filename = f"./../data/figures/maps/{row['id_veiculo_amostra']} {row['datetime_partida_amostra'].date()} {hora_formatada}.html"
        map.save(filename)

## (2) Gráficos da análise exploratória / análise histórica da linha


In [35]:
q = f"""
SELECT
    data, 
    sentido,
    COUNT(*) as quantidade_viagens
FROM
    `rj-smtr.projeto_subsidio_sppo.viagem_conformidade`
WHERE
    data BETWEEN '2022-06-01' AND '2023-09-15'
    AND servico_informado = '685'
GROUP BY
    data, sentido
ORDER BY
    data, sentido
"""

viagens_conformidade_ida_volta = bd.read_sql(q, from_file=True)
viagens_conformidade_ida_volta.info() 

Downloading: 100%|██████████| 906/906 [00:00<00:00, 2092.46rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 906 entries, 0 to 905
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   data                906 non-null    dbdate
 1   sentido             906 non-null    object
 2   quantidade_viagens  906 non-null    Int64 
dtypes: Int64(1), dbdate(1), object(1)
memory usage: 22.2+ KB


In [36]:
fig = px.line(viagens_conformidade_ida_volta, 
              x='data', 
              y='quantidade_viagens', 
              color='sentido', 
              title="Quantidade de Viagens Completas por Sentido")
fig.update_traces(opacity=0.45, line=dict(width=2))
fig.show()